## GradientTape의 활용

#### Automatic differentiation - GradientTape

Numpy만 가지고 딥러닝을 구현하는 것을 회상해 봅시다. model.fit()이라는 한 줄로 수행 가능한 딥러닝 모델 훈련 과정은 실제로는 어떠했나요?
  
1. Forward Propagation 수행 및 중간 레이어값 저장
2. Loss 값 계산
3. 중간 레이어값 및 Loss를 활용한 체인룰(chain rule) 방식의 역전파(Backward Propagation) 수행
4. 학습 파라미터 업데이트

이상 4단계로 이루어진 train_step 을 여러 번 반복했습니다.  
  
이런 과정이 TF2 API에는 model.fit()이라는 메서드 안에 모두 추상화되어 감추어져 있습니다.  

Tensorflow에서 제공하는 tf.GradientTape는 위와 같이 순전파(forward pass) 로 진행된 모든 연산의 중간 레이어값을 tape에 기록하고,   
이를 이용해 gradient를 계산한 후 tape를 폐기하는 기능을 수행합니다.   
그러면 아래에서는 이전 스텝에서 진행했던 학습을 tf.GradientTape를 이용한 것으로 변형해 보겠습니다.   
아래에서 할 tf.GradientTape는 이후 그래디언트를 좀 더 고급스럽게 활용하는 다양한 기법을 통해 자주 만나게 될 것입니다.

In [1]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

50000 10000
Metal device set to: Apple M1 Pro


여기까지는 앞에서 다루었던 Subclassing을 활용한 모델 작성법과 전혀 다르지 않습니다.
달라지는 것은 model.compile(), model.fit()을 통해 손쉽게 진행했던 학습 세팅 및 수행 부분입니다.



'''

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

'''

위와 같이 모델 학습을 위해 loss, optimizer를 지정해 주면 내부적으로는 매 스텝 학습이 진행될 때마다 발생하는 loss 및 그래디언트가 어떻게 학습 파라미터를 업데이트하게 되는지를 지정해 주는 작업이 model.compile() 안에서 자동으로 진행되었습니다.  

아래 코드는 tape.gradient()를 통해 매 스텝 학습이 진행될 때마다 발생하는 그래디언트를 추출한 후 optimizer.apply_gradients()를 통해 발생한 그래디언트가 업데이트해야 할 파라미터 model.trainable_variables를 지정해 주는 과정을 기술한 것입니다.

In [2]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.legacy.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

위와 같이 매 스텝 진행되는 학습의 실제 동작이 train_step() 메서드로 구현되었습니다.

 - model.fit(x_train, y_train, epochs=5, batch_size=32)  
    
이 model.fit()으로 위와 같이 한 줄로 간단하게 수행되던 실제 배치 학습 과정은,  
다름 아니라 매 스텝마다 위에서 구현했던 train_step()가 호출되는 과정으로 바꾸어 구현할 수 있습니다.  
model.fit() 호출 시에 결정되는 batch_size만 이번 스텝에서 결정해 주면 됩니다.

In [3]:
import numpy as np
import time

def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            x_batch.append(x)
            y_batch.append(y)
            if step % batch_size == batch_size-1:
                loss = train_step(np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.float32))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()

Epoch 0: last batch loss = 3.0454
Epoch 1: last batch loss = 2.5197
Epoch 2: last batch loss = 2.3166
Epoch 3: last batch loss = 2.1735
Epoch 4: last batch loss = 1.9544
It took 69.65213537216187 seconds


어떻습니까? 위에서 구현한 train_model() 메서드가 실은 우리가 그동안 사용했던 model.fit() 메서드와 기능적으로 같다는 것이 확인되시나요?  


이렇듯 tf.GradientTape()를 활용하면 model.compile()과 model.fit() 안에 감추어져 있던 한 스텝의 학습 단계(위 예제에서는 train_step 메서드)를 끄집어내서 자유롭게 재구성할 수 있게 됩니다.   

그동안 흔히 다루어 왔던 지도학습 방식과 다른 강화학습 또는 GAN(Generative Advasarial Network)의 학습을 위해서는 train_step 메서드의 재구성이 필수적이므로 tf.GradientTape()의 활용법을 꼭 숙지해 두셔야 합니다.

In [4]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test)  # Accuracy

1/1 [==============================] - 0s 95ms/step


2023-05-11 16:22:40.501926: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


0.3379

In [6]:
tf.config.set_visible_devices([], 'GPU')

RuntimeError: Visible devices cannot be modified after being initialized